In [1]:
import pandas as pd
import numpy as np
import opstrat as op
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import pandas_datareader.data as web  
from tqdm import tqdm
import math
import warnings 
from scipy.stats import norm

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sns.set(palette = 'viridis')

In [2]:
#данные предоставлены из доски на 9 апреля 2023 года

F0_sber = 216.1500
F0_lkoh = 4600
r = 0.0715     #дюрация 0.1 => процентная ставка 7,15% годовых (ставка ЦБ на 9 апреля 2023 года)
T = 38 / 365   #дней до экспирации опциона 

r_nsdq = 5.15371 / 100
T_nsdq = 41 / 365
F0_aapl = 175.43
F0_amzn = 120.11

df_sber = pd.read_csv('Data/SBER_clear_170523_090423.csv')
df_lkoh = pd.read_csv('Data/LKOH_clear_170523_090423.csv')
df_aapl = pd.read_csv('Data/AAPL_clear_07072023_27052023.csv')
df_amzn = pd.read_csv('Data/AMZN_clear_07072023_27052023.csv')


In [3]:
num_of_options_sber = df_sber.shape[0]
num_of_options_lkoh = df_lkoh.shape[0]
num_of_options_amzn = df_amzn.shape[0]
num_of_options_aapl = df_aapl.shape[0]

#### Ценообразование опционов симуляцией Монте-Карло

In [6]:
np.random.seed(1234)
# Параметры
T = T           # времени до даты экспирации
r = r           # безрисковая процентная ставка
q = 0.0         # дивидендная доходность актива в период между текущей датой и датой экспирации


def calculateMonteCarlo(df, T, r, q, S0, nsteps, npaths):
    prices = []
    num_of_options = df.shape[0]
    
    for i in tqdm(range(num_of_options)):

        # Параметры модели
        sigma = df['IV'][i] # подразумевая волатильность
        K = df['Strike'][i] # страйк


        # Риск-нейтральная мера
        muRN = r - q - 0.5 * sigma ** 2 # drift

        Vc_list = np.zeros(nsteps) 

        # Симуляция Монте-Карло
        for i in range(nsteps):
            X = muRN * T + sigma * np.sqrt(T) * np.random.normal(size=(1, npaths))
            S = S0 * np.exp(X)
            Vc_list[i] = np.exp(-r * T) * np.mean(np.maximum(S - K, 0))

        # Добавляем итоговую стоимость опциона в датафрейм
        Vc = np.mean(Vc_list)
        prices.append(Vc)

    df['MC Price'] = prices

In [7]:
calculateMonteCarlo(df_sber, T, r, S0 = F0_sber, q = 0, nsteps = 250, npaths = 20000)
calculateMonteCarlo(df_lkoh, T, r, S0 = F0_lkoh, q = 0, nsteps = 250, npaths = 20000)

100%|███████████████████████████████████████████| 31/31 [00:05<00:00,  5.90it/s]


In [8]:
calculateMonteCarlo(df_amzn, T = T_nsdq, r = r_nsdq, S0 = F0_amzn, q = 0, nsteps = 250, npaths = 20000)
calculateMonteCarlo(df_aapl, T = T_nsdq, r = r_nsdq, S0 = F0_aapl, q = 0, nsteps = 250, npaths = 20000)

100%|███████████████████████████████████████████| 40/40 [00:06<00:00,  6.18it/s]


In [9]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

print('MSE SBER: ', mean_squared_error(y_true = df_sber["Price"], 
                                  y_pred = df_sber["MC Price"]))
print('MAE SBER: ', mean_absolute_error(y_true = df_sber["Price"], 
                                  y_pred = df_sber["MC Price"]))

print('MSE LKOH: ', mean_squared_error(y_true = df_lkoh["Price"], 
                                  y_pred = df_lkoh["MC Price"]))
print('MAE LKOH: ', mean_absolute_error(y_true = df_lkoh["Price"], 
                                  y_pred = df_lkoh["MC Price"]))

MSE SBER:  0.06428162541473353
MAE SBER:  0.23937736969989068
MSE LKOH:  79.0527151476272
MAE LKOH:  8.726348237800224


#### Ценообразование опционов биномиальными деревьями

In [10]:
def Cox_Ross_Rubinstein_Tree(S, K, T, r, sigma, N, option_type):
    
    u = math.exp(sigma * math.sqrt(T / N));
    d = math.exp(-sigma * math.sqrt(T / N));
    pu = ((math.exp(r * T / N)) - d)/(u - d);
    pd = 1 - pu;
    disc = math.exp(-r * T / N);

    St = [0] * (N + 1)
    C = [0] * (N + 1)
    
    St[0] = S * d ** N;
    
    for j in range(1, N + 1): 
        St[j] = St[j - 1] * u / d;
    
    for j in range(1, N + 1):
        if option_type == 'P':
            C[j] = max(K - St[j], 0);
        elif option_type == 'C':
            C[j] = max(St[j] - K, 0);
    
    for i in range(N, 0, -1):
        for j in range(0, i):
            C[j] = disc * (pu * C[j+1] + pd * C[j]);
            
    return (C[0], St)

In [11]:
def Jarrow_Rudd_Tree(S, K, T, r, sigma, N, option_type):
    
    u = math.exp((r - (sigma ** 2 / 2)) * T / N + sigma * math.sqrt(T / N));
    d = math.exp((r - (sigma ** 2/2)) * T / N - sigma * math.sqrt(T / N));
    pu = 0.5;
    pd = 1 - pu;
    disc = math.exp(-r * T / N);

    St = [0] * (N + 1)
    C = [0] * (N + 1)
    
    St[0] = S * d ** N;
    
    for j in range(1, N + 1): 
        St[j] = St[j - 1] * u / d;
     
    for j in range(1, N + 1):
        if option_type == 'P':
            C[j] = max(K - St[j], 0);
        elif option_type == 'C':
            C[j] = max(St[j] - K, 0);
    
    for i in range(N, 0, -1):
        for j in range(0, i):
            C[j] = disc * (pu * C[j + 1] + pd * C[j]);
           
    return (C[0], St)

In [12]:
def calculateBinomialTree(df, T, r, S0, N, tree_type = 'Cox-Ross-Rubinstein'):
    num_of_options = df.shape[0]
    
    if tree_type == 'Cox-Ross-Rubinstein':
        bm_crr_prices = []
        for i in tqdm(range(num_of_options)):
            sigma = df['IV'][i] # волатильность 
            K = df['Strike'][i] # страйк
            current_price_crr = Cox_Ross_Rubinstein_Tree(S = S0, 
                                                         K = K,
                                                         T = T,
                                                         r = r,
                                                         sigma = sigma,
                                                         N = N,
                                                         option_type = 'C')
            bm_crr_prices.append(current_price_crr[0])
            
        df['BMCRR Price'] = bm_crr_prices
        
    if tree_type == 'Jarrow-Rudd':
        bm_jr_prices = []
        for i in tqdm(range(num_of_options)):
            sigma = df['IV'][i] # волатильность 
            K = df['Strike'][i] # страйк
            current_price_jr = Jarrow_Rudd_Tree(S = S0, 
                                                K = K,
                                                T = T,
                                                r = r,
                                                sigma = sigma,
                                                N = N,
                                                option_type = 'C')
            bm_jr_prices.append(current_price_jr[0])
        
        df['BMJR Price'] = bm_jr_prices

In [13]:
calculateBinomialTree(df_sber, T, r, S0 = F0_sber, N = 1000, tree_type = 'Cox-Ross-Rubinstein')
calculateBinomialTree(df_lkoh, T, r, S0 = F0_lkoh, N = 1000, tree_type = 'Cox-Ross-Rubinstein')

calculateBinomialTree(df_sber, T, r, S0 = F0_sber, N = 1000, tree_type = 'Jarrow-Rudd')
calculateBinomialTree(df_lkoh, T, r, S0 = F0_lkoh, N = 1000, tree_type = 'Jarrow-Rudd')

100%|███████████████████████████████████████████| 31/31 [00:03<00:00,  7.98it/s]


In [14]:
calculateBinomialTree(df_aapl, T = T_nsdq, r = r_nsdq, S0 = F0_aapl, N = 1000, tree_type = 'Cox-Ross-Rubinstein')
calculateBinomialTree(df_amzn, T = T_nsdq, r = r_nsdq, S0 = F0_amzn, N = 1000, tree_type = 'Cox-Ross-Rubinstein')

calculateBinomialTree(df_aapl, T = T_nsdq, r = r_nsdq, S0 = F0_aapl, N = 1000, tree_type = 'Jarrow-Rudd')
calculateBinomialTree(df_amzn, T = T_nsdq, r = r_nsdq, S0 = F0_amzn, N = 1000, tree_type = 'Jarrow-Rudd')

100%|███████████████████████████████████████████| 45/45 [00:05<00:00,  7.73it/s]


In [15]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

print('MSE SBER: ', mean_squared_error(y_true = df_sber["Price"], 
                                  y_pred = df_sber["BMCRR Price"]))
print('MAE SBER: ', mean_absolute_error(y_true = df_sber["Price"], 
                                  y_pred = df_sber["BMCRR Price"]))

print('MSE LKOH: ', mean_squared_error(y_true = df_lkoh["Price"], 
                                  y_pred = df_lkoh["BMCRR Price"]))
print('MAE LKOH: ', mean_absolute_error(y_true = df_lkoh["Price"], 
                                  y_pred = df_lkoh["BMCRR Price"]))

MSE SBER:  0.06362560748204074
MAE SBER:  0.23970951459041623
MSE LKOH:  78.46620709916454
MAE LKOH:  8.71284785608506


In [16]:
df_sber.to_csv('Results/sber_final_prices.csv', index = False)
df_lkoh.to_csv('Results/lkoh_final_prices.csv', index = False)
df_aapl.to_csv('Results/aapl_final_prices.csv', index = False)
df_amzn.to_csv('Results/amzn_final_prices.csv', index = False)

In [19]:
from itertools import product

models = ['MC Price', 'BMCRR Price', 'BMJR Price']
tickers = ['SBER', 'LKOH', 'AAPL', 'AMZN']
mse = pd.DataFrame({'Ticker': [i[1] for i in product(models, tickers)],
                    'Model': [i[0] for i in product(models, tickers)],
                   'MSE': [0]*12})
for col in models:
    mse['MSE'][(mse['Ticker'] == 'SBER') & (mse['Model'] == col)] = mean_squared_error(y_true = df_sber['Price'], y_pred = df_sber[col])
    mse['MSE'][(mse['Ticker'] == 'LKOH') & (mse['Model'] == col)] = mean_squared_error(y_true = df_lkoh['Price'], y_pred = df_lkoh[col])
    mse['MSE'][(mse['Ticker'] == 'AAPL') & (mse['Model'] == col)] = mean_squared_error(y_true = df_aapl['Price'], y_pred = df_aapl[col])
    mse['MSE'][(mse['Ticker'] == 'AMZN') & (mse['Model'] == col)] = mean_squared_error(y_true = df_amzn['Price'], y_pred = df_amzn[col])

In [20]:
from sklearn.metrics import r2_score
models = ['MC Price', 'BMCRR Price', 'BMJR Price']
tickers = ['SBER', 'LKOH', 'AAPL', 'AMZN']
r_square = pd.DataFrame({'Ticker': [i[1] for i in product(models, tickers)],
                    'Model': [i[0] for i in product(models, tickers)],
                   'R2': [0]*12})
for col in models:
    r_square['R2'][(r_square['Ticker'] == 'SBER') & (r_square['Model'] == col)] = (1 - r2_score(y_true = df_sber['Price'], y_pred = df_sber[col]))*100
    r_square['R2'][(r_square['Ticker'] == 'LKOH') & (r_square['Model'] == col)] = (1 - r2_score(y_true = df_lkoh['Price'], y_pred = df_lkoh[col]))*100
    r_square['R2'][(r_square['Ticker'] == 'AAPL') & (r_square['Model'] == col)] = (1 - r2_score(y_true = df_aapl['Price'], y_pred = df_aapl[col]))*100
    r_square['R2'][(r_square['Ticker'] == 'AMZN') & (r_square['Model'] == col)] = (1 - r2_score(y_true = df_amzn['Price'], y_pred = df_amzn[col]))*100
    

In [21]:
mse_new = mse.pivot_table(index='Ticker',
                    columns=mse.groupby('Ticker').cumcount() + 1,
                    values='MSE', 
                    aggfunc='first')\
        .add_prefix('Model')\
        .reset_index()\
        .rename(columns = {'Model1': 'MC Price', 'Model2': 'BMCRR Price', 'Model3': 'MBJR Price'})



In [22]:
r2_new = r_square.pivot_table(index='Ticker',
                    columns=r_square.groupby('Ticker').cumcount() + 1,
                    values='R2', 
                    aggfunc='first')\
        .add_prefix('Model')\
        .reset_index()\
        .rename(columns = {'Model1': 'MC Price', 'Model2': 'BMCRR Price', 'Model3': 'MBJR Price'})

In [23]:
mse_new

,Ticker,MC Price,BMCRR Price,MBJR Price
0,AAPL,0.004591,0.004820,0.004834
1,AMZN,0.000714,0.000571,0.000571
2,LKOH,79.052715,78.466207,78.556913
3,SBER,0.064282,0.063626,0.063241


In [24]:
r2_new

,Ticker,MC Price,BMCRR Price,MBJR Price
0,AAPL,0.000271,0.000285,0.000286
1,AMZN,0.000296,0.000236,0.000237
2,LKOH,0.037128,0.036852,0.036895
3,SBER,0.008218,0.008134,0.008085
